# CRUD Operations with SQLite3 and Python

SQLite is a lightweight, serverless, self-contained SQL database engine that comes built-in with Python. It's perfect for:
- Small to medium-sized applications
- Desktop applications
- Prototyping and testing
- Data analysis and storage
- Mobile applications

In this notebook, we'll learn how to perform **CRUD** operations (Create, Read, Update, Delete) using SQLite3 with Python.

## What is CRUD?

CRUD stands for the four basic operations performed on databases:

| Operation | Description | SQL Command |
|-----------|-------------|-------------|
| **C**reate | Insert new records | INSERT INTO |
| **R**ead | Retrieve/Query records | SELECT |
| **U**pdate | Modify existing records | UPDATE |
| **D**elete | Remove records | DELETE |

## Why SQLite3?

- **Zero Configuration**: No server setup required
- **Built-in**: Comes with Python standard library
- **Portable**: Database is a single file
- **Lightweight**: Small memory footprint
- **ACID Compliant**: Reliable transactions

---

## 1. Importing SQLite3 and Connecting to Database

The `sqlite3` module is part of Python's standard library, so no installation is needed.

In [ ]:
import sqlite3

# Connect to database (creates file if it doesn't exist)
connection = sqlite3.connect('students.db')

# Create a cursor object to execute SQL commands
cursor = connection.cursor()

print("Database connected successfully!")
print(f"SQLite version: {sqlite3.version}")

## 2. Creating a Table (CREATE)

Let's create a `students` table with the following structure:
- `id` (Primary Key, Auto-increment)
- `name` (Text)
- `age` (Integer)
- `grade` (Text)
- `email` (Text)

In [ ]:
# Create table SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    grade TEXT NOT NULL,
    email TEXT UNIQUE
)
"""

# Execute the query
cursor.execute(create_table_query)

# Commit the changes
connection.commit()

print("Table 'students' created successfully!")

---

## 3. Inserting Data (CREATE)

There are several ways to insert data into a table:
1. Insert single record
2. Insert multiple records
3. Using parameters for security (prevent SQL injection)

In [ ]:
# Method 1: Insert a single record (AVOID - SQL Injection risk)
# cursor.execute("INSERT INTO students (name, age, grade, email) VALUES ('John Doe', 20, 'A', 'john@example.com')")

# Method 2: Insert using parameters (RECOMMENDED - Safe from SQL injection)
insert_query = "INSERT INTO students (name, age, grade, email) VALUES (?, ?, ?, ?)"
student_data = ("Alice Johnson", 19, "A", "alice@example.com")

cursor.execute(insert_query, student_data)
connection.commit()

print(f"Record inserted! Row ID: {cursor.lastrowid}")

### Inserting Multiple Records

In [ ]:
# Insert multiple records using executemany()
students_list = [
    ("Bob Smith", 21, "B", "bob@example.com"),
    ("Charlie Brown", 20, "A", "charlie@example.com"),
    ("Diana Prince", 22, "A+", "diana@example.com"),
    ("Eve Wilson", 19, "B+", "eve@example.com"),
    ("Frank Miller", 23, "C", "frank@example.com")
]

cursor.executemany(insert_query, students_list)
connection.commit()

print(f"Inserted {cursor.rowcount} records successfully!")

---

## 4. Reading Data (READ)

We can retrieve data using `SELECT` statements. Common methods:
- `fetchone()` - Retrieves one record
- `fetchall()` - Retrieves all records
- `fetchmany(n)` - Retrieves n records

In [ ]:
# Fetch all records
cursor.execute("SELECT * FROM students")
all_students = cursor.fetchall()

print("All Students:")
print("-" * 70)
for student in all_students:
    print(f"ID: {student[0]}, Name: {student[1]}, Age: {student[2]}, Grade: {student[3]}, Email: {student[4]}")

### Fetch with Conditions

In [ ]:
# Fetch students with grade 'A' or 'A+'
cursor.execute("SELECT * FROM students WHERE grade LIKE 'A%'")
top_students = cursor.fetchall()

print("\nTop Students (Grade A):")
print("-" * 70)
for student in top_students:
    print(f"Name: {student[1]}, Age: {student[2]}, Grade: {student[3]}")

### Using Row Factory for Dictionary-like Access

In [ ]:
# Set row_factory to access columns by name
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

cursor.execute("SELECT * FROM students WHERE age < 21")
young_students = cursor.fetchall()

print("\nStudents Under 21 (using row factory):")
print("-" * 70)
for student in young_students:
    print(f"Name: {student['name']}, Age: {student['age']}, Email: {student['email']}")

---

## 5. Updating Data (UPDATE)

Update existing records in the database.

In [ ]:
# Update a student's grade
update_query = "UPDATE students SET grade = ? WHERE name = ?"
cursor.execute(update_query, ("A+", "Bob Smith"))
connection.commit()

print(f"Updated {cursor.rowcount} record(s)")

# Verify the update
cursor.execute("SELECT * FROM students WHERE name = 'Bob Smith'")
updated_student = cursor.fetchone()
print(f"\nUpdated Record: Name: {updated_student['name']}, Grade: {updated_student['grade']}")

### Update Multiple Records

In [ ]:
# Increase age by 1 for all students
cursor.execute("UPDATE students SET age = age + 1")
connection.commit()

print(f"Updated {cursor.rowcount} student(s) ages")

# Display updated ages
cursor.execute("SELECT name, age FROM students")
for student in cursor.fetchall():
    print(f"{student['name']}: {student['age']} years old")

---

## 6. Deleting Data (DELETE)

Remove records from the database.

In [ ]:
# Delete a specific student
delete_query = "DELETE FROM students WHERE name = ?"
cursor.execute(delete_query, ("Frank Miller",))
connection.commit()

print(f"Deleted {cursor.rowcount} record(s)")

# Verify deletion
cursor.execute("SELECT COUNT(*) FROM students")
count = cursor.fetchone()[0]
print(f"Total students remaining: {count}")

---

## 7. Transactions and Error Handling

Transactions ensure data integrity. Use `try-except` blocks to handle errors gracefully.

In [ ]:
try:
    # Start a transaction
    cursor.execute("BEGIN TRANSACTION")
    
    # Try to insert a duplicate email (will fail due to UNIQUE constraint)
    cursor.execute("INSERT INTO students (name, age, grade, email) VALUES (?, ?, ?, ?)",
                  ("Test User", 20, "A", "alice@example.com"))  # Duplicate email
    
    connection.commit()
    print("Transaction successful!")
    
except sqlite3.IntegrityError as e:
    # Rollback if there's an error
    connection.rollback()
    print(f"Error: {e}")
    print("Transaction rolled back!")
    
except Exception as e:
    connection.rollback()
    print(f"Unexpected error: {e}")